<a href="https://colab.research.google.com/github/StickMonkey615/JHCSMod4/blob/main/Final%20Classifier%20Model%20v3.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Module 4 Guidance

This notebook is a template for module 4b and 4c, which will be tested in Google Colab, your code needs to run there.
The structure has been provided to improve consistency and make it easier for markers to understand your code but still give students the flexibility to be creative.  You need to populate the required functions to solve this problem.  All dependencies should be documented in the next cell.

You can:
    add further cells or text blocks to extend or further explain your solution
    add further functions

Dont:
    rename functions
   

In [1]:
# Fixed dependencies - do not remove or change.
import pytest
import pandas as pd
import numpy as np
from google.colab import drive
# drive.mount('/content/gdrive/')
# Import your dependencies
!pip install --upgrade xlrd > 1.2.0
!pip install imbalanced-learn
!pip install --upgrade imbalanced-learn
import xlrd
import imblearn
import matplotlib.pyplot as plt

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
  Using cached imbalanced_learn-0.9.1-py3-none-any.whl (199 kB)


In [2]:
# Import data

def import_local_data(file_path):
    """This function needs to import the data file into collab and return a pandas dataframe
    """
    raw_df = pd.read_excel(file_path)
    return raw_df

In [3]:
local_file_path = "breast-cancer.xls"

In [4]:
# Dont change
raw_data = import_local_data(local_file_path)

### Conduct exploratory data analysis and explain your key findings - Examine the data, explain its key features and what they look like.  Highlight any fields that are anomalous.

In [5]:
# Look at the different dataframe column headings
print(raw_data.columns)

Index(['age', 'menopause', 'tumor-size', 'inv-nodes', 'node-caps', 'deg-malig',
       'breast', 'breast-quad', 'irradiat', 'Class'],
      dtype='object')


In [6]:
# Determine data types for each column
for i in range(0, len(raw_data.columns)):
    print(type(raw_data.values[1][i]))

<class 'str'>
<class 'str'>
<class 'str'>
<class 'str'>
<class 'str'>
<class 'int'>
<class 'str'>
<class 'str'>
<class 'str'>
<class 'str'>


In [7]:
#Look at the range of values for each field
from collections import Counter
rng_vals=[]
for i in range(0,len(raw_data.columns)):
    rng_vals.append(Counter(raw_data.iloc[:,i].values))
    print(f"{raw_data.columns[i]}: {rng_vals[i]}")
del rng_vals, i

age: Counter({'50-59': 96, '40-49': 90, '60-69': 57, '30-39': 36, '70-79': 6, '20-29': 1})
menopause: Counter({'premeno': 150, 'ge40': 129, 'lt40': 7})
tumor-size: Counter({'30-34': 60, '25-29': 54, '20-24': 50, '15-19': 30, datetime.datetime(2014, 10, 1, 0, 0): 28, '40-44': 22, '35-39': 19, '0-4': 8, '50-54': 8, datetime.datetime(2019, 9, 5, 0, 0): 4, '45-49': 3})
inv-nodes: Counter({'0-2': 213, datetime.datetime(2019, 5, 3, 0, 0): 36, datetime.datetime(2019, 8, 6, 0, 0): 17, datetime.datetime(2019, 11, 9, 0, 0): 10, '15-17': 6, datetime.datetime(2014, 12, 1, 0, 0): 3, '24-26': 1})
node-caps: Counter({'no': 222, 'yes': 56, '?': 8})
deg-malig: Counter({2: 130, 3: 85, 1: 71})
breast: Counter({'left': 152, 'right': 134})
breast-quad: Counter({'left_low': 110, 'left_up': 97, 'right_up': 33, 'right_low': 24, 'central': 21, '?': 1})
irradiat: Counter({'no': 218, 'yes': 68})
Class: Counter({'no-recurrence-events': 201, 'recurrence-events': 85})


**All fields look to contain data that is catagorical in nature.**

**Some contain data that appears erroneous:**
 
*   **'tumor-size' and 'inv-nodes' appear to contain some data in a datetime format and some in string.**
*   **'node-caps' and 'breast-quad' contain Question Marks.**

**Need a way to address these erroneous data inputs.**



In [8]:
# Look in more detail at the columns with datetime data.
print(raw_data.iloc[:, 2].values)

['15-19' '15-19' '35-39' '35-39' '30-34' '25-29' '40-44'
 datetime.datetime(2014, 10, 1, 0, 0) '0-4' '40-44' '25-29' '15-19'
 '30-34' '25-29' '25-29' '20-24' datetime.datetime(2014, 10, 1, 0, 0)
 '15-19' '40-44' '20-24' '20-24' '40-44' '15-19'
 datetime.datetime(2014, 10, 1, 0, 0) '15-19' '20-24'
 datetime.datetime(2014, 10, 1, 0, 0) datetime.datetime(2014, 10, 1, 0, 0)
 '30-34' '15-19' '30-34' '25-29' '25-29' '20-24' '30-34' '15-19'
 datetime.datetime(2014, 10, 1, 0, 0) '45-49' '20-24'
 datetime.datetime(2014, 10, 1, 0, 0) '35-39' '35-39' '25-29' '20-24'
 '15-19' '30-34' datetime.datetime(2014, 10, 1, 0, 0) '35-39' '50-54'
 '40-44' '15-19' '30-34' '0-4' '40-44' '25-29' '25-29' '20-24' '35-39'
 '50-54' '0-4' '40-44' '30-34' '20-24' '30-34' '20-24' '15-19' '25-29'
 '15-19' '50-54' datetime.datetime(2014, 10, 1, 0, 0) '25-29' '25-29'
 datetime.datetime(2014, 10, 1, 0, 0) '30-34' '25-29'
 datetime.datetime(2014, 10, 1, 0, 0) '15-19' '25-29' '25-29' '30-34'
 '15-19' '25-29' '30-34' '15-19'

In [9]:
# Look at output data
#print(raw_data.iloc[:, -1].values)
raw_data['Class'].value_counts()

no-recurrence-events    201
recurrence-events        85
Name: Class, dtype: int64

Only 2 possible outputs, thus needs converting to binary format for use in classifier models.

In [10]:
# Check output balance
out = raw_data.iloc[:, -1].values
no_rows = len(raw_data)

from sklearn.preprocessing import LabelEncoder
le = LabelEncoder()
code_rows = le.fit_transform(out)
print("True outputs: {:.2f} %".format(sum(code_rows)/no_rows*100))
del out, no_rows, le, code_rows

True outputs: 29.72 %


Clear imbalance between output data. Some degree of bias/weighting/sampling will be required to ensure that results accurately predict outcomes for both True and False outcomes.

In [11]:
# Explain your key findings

**Data set is made up of 9 independent variables all of which appear catagorical in nature. Although stored as an integer, 'deg-malig' can be viewed as  catagorical data as it can only contain 3 discrete values.**

**The inclusion of datetime data entries in both the 'tumor-size' and 'inv-nodes' fields appears to be caused by a formatting entry within Excel. For example, '10-14' being input erroneously as 10/14 thus Excel has interpreted (and converted) it to the datetime field 01/10/2014. A function will need to be written within the model to convert these back to correct format.**

**How to deal with '?' entries in fields that are otherwise boolean poses an interesting dilemma. If these are infact meant to signify that the presence is unknown because no diagnostic work has been conducted, then this woiuld signify a valid data entry. If it is however just an incomplete data entry then there is a risk its inclusion could skew the model results. Without knowing which it seems wisest to remove this data from the dataset. Removal of the entire field could well deprive the model of important information, thus just removing these specific entries (rows) appears the most sensible option, particularly noting that there are relatively few occurences.**

**Data set is imbalanced, with dependent variable outputs only True in 30% of instances. The model applied will require this imbalance to be taken into account so as not to sacrifice results predicting this smaller class (surely the aim of cancer diagnosis) so as to achieve a high accuracy figure.**

**Output variable will need converting into binary output for use with a binary classification model.**

Create any data pre-processing that you will conduct on seen and unseen data.  Regardless of the model you use, this dataframe must contain only numeric features and have a strategy for any expected missing values. Any objects can that are needed to handle the test data that are dependent on the training data can be stored in the model class.  You are recommended to use sklearn Pipelines or similar functionality to ensure reproducibility.

In [12]:
class Module4_Model:
    
    def __init__(self):
        self.model = None

    def process_dataframe(self, unproc_data):
        """
        Function to remove any erroneous data and then process all data into a
        Pandas Dataframe with all data converted into catagorical date (i.e. 
        encoded) and dummy variables dropped.
        """
        # Deal with ? entries (remove them)
        indx = unproc_data[unproc_data.isin(['?'])].stack(dropna=True).unstack().index
        print(f"indx: {indx}")
        unproc_data = unproc_data.drop(index=indx)

        # Remove non-categorical data
        dm = unproc_data.pop('deg-malig')

        # Encode the catagorical data (dummy variables)
        proc_data = pd.get_dummies(data=unproc_data, prefix_sep='_', drop_first=True)
    
        # Add back in non-categorical data
        proc_data.insert(0, 'deg-malig', dm)
    
        return proc_data
    
    def reformat_dates(self, data):
        """ 
        Function to reformat datetime data entries in 'tumor-size' and 
        'inv-nodes' fields back into the string format expected.
        """
        # Correct date types in 'tumor-size' and 'inv-nodes' variables
        for i in range(0, len(data)):
            if type(data['tumor-size'][i]) is not str:
                if data['tumor-size'][i].day == 1:
                    data['tumor-size'][i] = str(data['tumor-size'][i].month) +'-' + str(data['tumor-size'][i].year-2000)
                else:
                    data['tumor-size'][i] = str(data['tumor-size'][i].day) + '-' + str(data['tumor-size'][i].month)
            if type(data['inv-nodes'][i]) is not str:
                if data['inv-nodes'][i].day == 1:
                    data['inv-nodes'][i] = str(data['inv-nodes'][i].month) + '-' + str(data['inv-nodes'][i].year-2000)
                else:
                    data['inv-nodes'][i] = str(data['inv-nodes'][i].day) + '-' + str(data['inv-nodes'][i].month)
        
        return data

    def preprocess_training_data(self, training_df):
        """
        This function should process the training data and store any features
        required in the class
        """
        # Apply feature scaling
        from sklearn.preprocessing import StandardScaler
        sc = StandardScaler()
        processed_df = sc.fit_transform(training_df)
        return processed_df

    def preprocess_test_data(self, test_df):
        """
        This function should process the test data and store any features
        required in the class
        """
        # Apply feature scaling
        from sklearn.preprocessing import StandardScaler
        sc = StandardScaler()
        processed_df = sc.fit_transform(test_df)
        return processed_df



In [13]:
# Dont change
my_model = Module4_Model()
# reformat date fields into string
raw_data_date = my_model.reformat_dates(raw_data)

# clean up data and create dataframe of only numerical (catagorical) data
clean_data = my_model.process_dataframe(raw_data_date)
del raw_data_date, raw_data

indx: Int64Index([20, 31, 50, 54, 71, 92, 149, 240, 264], dtype='int64')


/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:44: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:37: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:39: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:42: SettingWithCopyWarning: 
A value is trying to b

In [14]:
# Split your data so that you can test the effectiveness of your model
# Split the data into a Training set and a Test set
dfs = np.split(clean_data, [len(clean_data.columns)-1], axis=1)
X = dfs[0]
y = dfs[1]
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X,
                                                    y,
                                                    test_size = 0.25,
                                                    random_state = 42)
del dfs, X, y, clean_data

In [15]:
# Dont change
x_train_processed = my_model.preprocess_training_data(X_train)
del X_train

In [16]:
# Populate preprocess_training_data and preprocess_test_data to preprocess data.
# You must process test and train separately so your model does not accidently
# gain information that a model wouldnt have in reality and therefore get better predictions


In [17]:
# Dont change
x_test_processed = my_model.preprocess_test_data(X_test)
del X_test

In [18]:
# Apply dimensionality reduction
from sklearn.decomposition import KernelPCA
kpca = KernelPCA(n_components = 8, kernel = 'rbf')
x_train_processed = kpca.fit_transform(x_train_processed)
x_test_processed = kpca.transform(x_test_processed)

In [19]:
# Create models for evaluation
from sklearn.ensemble import AdaBoostClassifier
from sklearn.ensemble import GradientBoostingClassifier
from imblearn.ensemble import EasyEnsembleClassifier, BalancedRandomForestClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.svm import SVC
from sklearn.linear_model import LogisticRegression
from sklearn.experimental import enable_halving_search_cv
from sklearn.model_selection import HalvingGridSearchCV
from sklearn.metrics import accuracy_score,log_loss,recall_score,balanced_accuracy_score
from sklearn.metrics import precision_score,f1_score,confusion_matrix, fbeta_score
from imblearn.metrics import sensitivity_score, specificity_score, geometric_mean_score

models = ['ADA',
          'GBC',
          'EEC',
          'BRF',
          'RFC',
          'KNN',
          'SVC',
          'lReg']

classifiers = [AdaBoostClassifier(random_state=42),
               GradientBoostingClassifier(random_state=42),
               EasyEnsembleClassifier(base_estimator=RandomForestClassifier(random_state=42,
                                                                            n_estimators=60)),
               BalancedRandomForestClassifier(random_state=42),
               RandomForestClassifier(random_state=42, n_jobs=-1),
               KNeighborsClassifier(n_jobs=-1),
               SVC(random_state=42, probability=True),
               LogisticRegression(solver='newton-cg', multi_class='multinomial')]


params = {models[0]:{'learning_rate':[0.01,0.1,1,10],
                     'n_estimators':np.array(range(10,110,10)),
                     'algorithm':['SAMME','SAMME.R']},
          models[1]:{'learning_rate':[0.01,0.1,1,10],
                     'n_estimators':np.array(range(10,110,10)),
                     'max_depth':np.array(range(1,11,1))},
          models[2]:{'n_estimators':np.array(range(10,60,10)),
                     'base_estimator__n_estimators':np.array(range(40,100,10)),
                     'base_estimator__criterion':['gini','entropy'],
                     'base_estimator__class_weight':['balanced','balanced_subsample']},
          models[3]:{'n_estimators':np.array(range(10,110,10)),
                     'criterion':['gini','entropy'],
                     'class_weight':['balanced','balanced_subsample']},
          models[4]:{'n_estimators':np.array(range(10,110,10)),
                     'criterion':['gini','entropy'],
                     'class_weight':['balanced','balanced_subsample']},
          models[5]:{'n_neighbors':np.array(range(10,60,10)),
                     'weights':['uniform','distance'],
                     'algorithm':['auto','ball_tree'],
                     'metric':['chebyshev','minkowski']},
          models[6]:{'C':[1,10,60,100,600,1000],
                     'tol':[0.005],
                     'kernel':['linear','poly','rbf','sigmoid'],
                     'class_weight':['balanced']},
          models[7]:{'C':[1,10,60,100,600,1000],
                     'class_weight':['balanced'],
                     'solver':['newton-cg','lbfgs','sag','saga'],
                     'tol':[0.0001]}}

In [20]:
# Define custom scoring metric
from sklearn.metrics import make_scorer
import math
def my_scorer(y_true, y_pred):
    score = accuracy_score(y_true,y_pred) * f1_score(y_true,y_pred,average='macro')
    if math.isnan(score):
        score = 0.01
    return score

In [ ]:
cust_score = make_scorer(my_scorer,greater_is_better=True)
scorers_list = [geometric_mean_score,
                fbeta_score,
                'accuracy',
                'precision',
                'recall',
                'f1_macro',
                cust_score,
                'balanced_accuracy',
                sensitivity_score,]

from google.colab import files
for i in range(2,len(scorers_list)):
    # Conduct halving grid search across all models
    y_tested=0
    test_scores=[]
    for name, estimator in zip(models,classifiers):
        print(name)
        clf = HalvingGridSearchCV(estimator=estimator,
                                param_grid=params[name],
                                factor=2,
                                scoring=scorers_list[i],
                                cv=5,
                                n_jobs=-1)
        clf.fit(x_train_processed, np.ravel(y_train.values))
        print(f"Best params: {clf.best_params_}")
        print(f"Best score: {clf.best_score_}")
        estimates = clf.predict_proba(x_test_processed)
        y_tested+=estimates
        acc = accuracy_score(y_test, clf.predict(x_test_processed))
        rec = recall_score(y_test, clf.predict(x_test_processed))
        pre = precision_score(y_test, clf.predict(x_test_processed))
        f1s = f1_score(y_test, clf.predict(x_test_processed), average='macro')
        cm = confusion_matrix(y_test, clf.predict(x_test_processed))
        print("Accuracy: {:.4%}".format(acc))
        print("Recall: {:.4%}".format(rec))
        print("Precision: {:.4%}".format(pre))
        print("F-measure: {:.4%}".format(f1s))
        print(cm)
        
        test_scores.append((name,acc,clf.best_score_,f1s,rec,pre,cm,clf.best_params_,(0.5*(f1s+acc))))
        
    submission = pd.DataFrame(test_scores, columns=['Classifier',
                                                    'Accuracy',
                                                    'Trg score',
                                                    'F-score test',
                                                    'Recall',
                                                    'Precision',
                                                    'Confusion matrix',
                                                    'Best params',
                                                    'Custom score'])
    submission.to_csv(f"Results_{i}.csv")
    files.download(f"/content/Results_{i}.csv")

ADA
Best params: {'algorithm': 'SAMME.R', 'learning_rate': 0.01, 'n_estimators': 60}
Best score: 0.7203629032258065
Accuracy: 72.8571%
Recall: 0.0000%
Precision: 0.0000%
F-measure: 42.1488%
[[51  0]
 [19  0]]
GBC


/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


Best params: {'learning_rate': 0.01, 'max_depth': 8, 'n_estimators': 10}
Best score: 0.6941532258064517
Accuracy: 72.8571%
Recall: 0.0000%
Precision: 0.0000%
F-measure: 42.1488%
[[51  0]
 [19  0]]
EEC


/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


Best params: {'base_estimator__class_weight': 'balanced_subsample', 'base_estimator__criterion': 'entropy', 'base_estimator__n_estimators': 80, 'n_estimators': 20}
Best score: 0.6235887096774193
Accuracy: 57.1429%
Recall: 84.2105%
Precision: 37.2093%
F-measure: 56.5757%
[[24 27]
 [ 3 16]]
BRF
Best params: {'class_weight': 'balanced', 'criterion': 'gini', 'n_estimators': 90}
Best score: 0.6235887096774193
Accuracy: 57.1429%
Recall: 84.2105%
Precision: 37.2093%
F-measure: 56.5757%
[[24 27]
 [ 3 16]]
RFC
Best params: {'class_weight': 'balanced_subsample', 'criterion': 'entropy', 'n_estimators': 100}
Best score: 0.7328629032258066
Accuracy: 64.2857%
Recall: 26.3158%
Precision: 31.2500%
F-measure: 52.3810%
[[40 11]
 [14  5]]
KNN


...

/usr/local/lib/python3.7/dist-packages/sklearn/model_selection/_search.py:972: UserWarning: One or more of the test scores are non-finite: [0.2 0.2 nan nan nan nan nan nan nan nan 0.2 0.2 nan nan nan nan nan nan
 nan nan 0.2 0.2 nan nan nan nan nan nan nan nan 0.2 0.2 nan nan nan nan
 nan nan nan nan]
  category=UserWarning,
/usr/local/lib/python3.7/dist-packages/sklearn/model_selection/_search.py:972: UserWarning: One or more of the train scores are non-finite: [0.125 1.      nan   nan   nan   nan   nan   nan   nan   nan 0.125 1.
   nan   nan   nan   nan   nan   nan   nan   nan 0.125 1.      nan   nan
   nan   nan   nan   nan   nan   nan 0.125 1.      nan   nan   nan   nan
   nan   nan   nan   nan]
  category=UserWarning,
/usr/local/lib/python3.7/dist-packages/sklearn/model_selection/_search.py:972: UserWarning: One or more of the test scores are non-finite: [0.2        0.2               nan        nan        nan        nan
        nan        nan        nan        nan 0.2        0.2
 

Best params: {'algorithm': 'auto', 'metric': 'chebyshev', 'n_neighbors': 40, 'weights': 'distance'}
Best score: 0.09444444444444444
Accuracy: 71.4286%
Recall: 0.0000%
Precision: 0.0000%
F-measure: 41.6667%
[[50  1]
 [19  0]]
SVC
Best params: {'C': 1, 'class_weight': 'balanced', 'kernel': 'poly', 'tol': 0.005}
Best score: 0.8177777777777777
Accuracy: 42.8571%
Recall: 84.2105%
Precision: 30.1887%
F-measure: 42.8105%
[[14 37]
 [ 3 16]]
lReg
Best params: {'C': 1, 'class_weight': 'balanced', 'solver': 'lbfgs', 'tol': 0.0001}
Best score: 0.6697474747474748
Accuracy: 64.2857%
Recall: 84.2105%
Precision: 42.1053%
F-measure: 63.0099%
[[29 22]
 [ 3 16]]


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

ADA
Best params: {'algorithm': 'SAMME', 'learning_rate': 1, 'n_estimators': 80}
Best score: 0.6158071215392866
Accuracy: 65.7143%
Recall: 36.8421%
Precision: 36.8421%
F-measure: 56.6563%
[[39 12]
 [12  7]]
GBC
Best params: {'learning_rate': 1, 'max_depth': 10, 'n_estimators': 100}
Best score: 0.5758487456584848
Accuracy: 71.4286%
Recall: 26.3158%
Precision: 45.4545%
F-measure: 57.5758%
[[45  6]
 [14  5]]
EEC


In [ ]:
# Select the highest performing classifier
indx = submission['Trg score'].idxmax()
my_model.classifier = classifiers[indx].set_params(**submission['Best params'][indx])

# Train your model
my_model.classifier.fit(x_train_processed, np.ravel(y_train.values))

In [ ]:
# use your model to make a prediction on unseen data
y_pred = my_model.classifier.predict(x_test_processed)

In [ ]:
# Asssess the accuracy of your model and explain your key findings
# Generate confusion matrix
from sklearn.metrics import confusion_matrix, accuracy_score, balanced_accuracy_score, roc_auc_score, ConfusionMatrixDisplay
cm = confusion_matrix(y_test, y_pred, labels=my_model.classifier.classes_)
print("Accuracy: {:.2f} %".format(accuracy_score(y_test, y_pred)*100))
print("Weighted ROC AUC accuracy: {:.2f} %".format(roc_auc_score(y_test, y_pred, average='weighted')*100))
print("Confusion matrix:")
disp = ConfusionMatrixDisplay(cm, display_labels=my_model.classifier.classes_)
disp.plot()
plt.show()
print("False negatives: {:.2f} %".format(cm[1,0]/sum(sum(cm))*100))
print("F-score (macro): {:.2f} %".format(f1_score(y_test,y_pred,average='macro')*100))

# Apply k-fold Cross Validation
from sklearn.model_selection import cross_val_score
accuracies = cross_val_score(estimator=my_model.classifier,
                             X=x_train_processed,
                             y=np.ravel(y_train.values),
                             scoring='roc_auc_ovo',
                             cv=10)
print("K-fold cross validation results")
print("Accuracy: {:.2f} %".format(accuracies.mean()*100))
print("Standard Deviation: {:.2f} %".format(accuracies.std()*100))
del cm, accuracies, disp

**Model has an accuracy of just over 70% when predicting whether cancer recurrence will occur.**

**Crucially, the proportion of False Negatives is low (c.10%). In cancer diagnosis these are the outcomes that we want to minimise. False Positives, whilst undesirable, will likely lead to further diagnostic testing before it is realised that cancer is not present.**

### Unit tests:

# New section

###Checking training and test data for null values. This will work for both pd dataframes and np arrays, and ensures no null values exist.

In [ ]:
def test_no_nulls(data):
    """ Assert no null values within pd dataframe or np array """
    
    # if data is numpy array, handle accordingly
    if isinstance(data, (np.ndarray)):
        assert not np.isnan(np.min(data))
    
    # if not np array, assume data is pandas dataframe
    else:
        assert data.isna().sum().sum() == 0

In [ ]:
# run null data unit test on both training and test data
test_no_nulls(x_train_processed)
test_no_nulls(x_test_processed)